# experiments with Agents

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.tools import BaseTool
from langchain.agents import AgentType, initialize_agent
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from pydantic import Field
from typing import Optional, Type

In [ ]:
class BoxStorage():
    def __init__(self, contents):
        """ contents is something like {box -> set-of-strings }"""
        self.boxes = contents
    
    def list_boxes(self): return list(self.boxes.keys())
    
    def list_box(self, box): return list(self.boxes[box])

    def move_objects(self, objs, bfrom, bto):
        for obj in objs:
            assert(obj in self.boxes[bfrom])
            self.boxes[bfrom] = self.boxes[bfrom] - {obj}
            self.boxes[bto] = self.boxes[bto] | {obj}

    def __repr__(self):
        return 'BoxStorage[%s]' % (', '.join(
            '%s=(%s)' % (k, ', '.join(sorted(v)))
            for k, v in sorted(self.boxes.items())
        ))

In [ ]:
class CustomListBoxesTool(BaseTool):
    name = "list_boxes"
    description = "The tool to get a comma-separated list of available boxes. The input is ignored."
    storage: BoxStorage = Field(exclude=True)
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return ', '.join(self.storage.list_boxes())
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("no async")

class CustomMoveObjectsTool(BaseTool):
    name = "move_object"
    description = """This is the tool you use to physically move a space-separated list of objects from a box to another box.
    The input must have the form "MOVE object_name_1 object_name_2 ... FROM source_box TO destination_box".
    Each invocation can move a number of objects to exactly one box. Moving to multiple boxes requires mu"""
    storage: BoxStorage = Field(exclude=True)
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        # an ugly syntax validation, lol
        qchunks = [c.strip() for c in query.split(' ') if c.strip()]
        fromIndex = [i for i, c in enumerate(qchunks) if c=='FROM'][0]
        assert(qchunks[0].lower() == 'move')
        assert(qchunks[fromIndex].lower() == 'from')
        assert(qchunks[2 + fromIndex].lower() == 'to')
        assert(len(qchunks) == 4 + fromIndex)
        obs_to_move = set(qchunks[1:fromIndex])
        bfrom = qchunks[1 + fromIndex]
        bto = qchunks[3 + fromIndex]
        self.storage.move_objects(obs_to_move, bfrom, bto)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("no async")

class CustomListBoxContentTool(BaseTool):
    name = "box_contents"
    description = "The tool to get a comma-separated list of contents of a box. The input is the box name."
    storage: BoxStorage = Field(exclude=True)

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return ', '.join(self.storage.list_box(query))
    
    async def _arun(self, query: str,  run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("no async")


## A simple, guided case

In [ ]:
storage = BoxStorage({'box1': {'apple','banana','pear'}, 'box2': {'car', 'bicycle', 'train'}})
storage

In [ ]:
tools = [
    CustomListBoxesTool(storage=storage),
    CustomListBoxContentTool(storage=storage),
    CustomMoveObjectsTool(storage=storage)
]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("""
    You are given some named boxes, each containing some named items,
    and the tools to inspect and operate on them.
    Verify if there are pears or apples in box1,
    then move all of them to box2. You can move several objects at once,
    if that is permitted by the available tools.
    Stop when there are neither pears nor apples in box1.
""")

In [ ]:
storage

## A more 'semantic' experiment

In [ ]:
storage2 = BoxStorage({'fruit': {'apple', 'dog'}, 'animals': {'mango', 'cheetah'}})
storage2

In [ ]:
tools2 = [
    CustomListBoxesTool(storage=storage2),
    CustomListBoxContentTool(storage=storage2),
    CustomMoveObjectsTool(storage=storage2)
]
agent2 = initialize_agent(tools2, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent2.run("""
    You are given some boxes, each containing some items,
    and the tools to inspect and operate on them.
    
    The name of each box describes which kind of objects the box is supposed to contain:
    for example, the "countries" box should contain Italy and Sweden, but not Rome, bicycle or Saturn.
    A box should not contain other types of objects.
    
    Your task is the following:
    - first survey each box to figure out whether there are misplaced items in any of them;
    - then, by moving misplaced items to the correct boxes, establish order.
    - Finish only after having thoroughly and carefully verified that each box contains only things that are
    supposed to be there.
""")

In [ ]:
storage2

### Note the agent starts to get confused if there are more items and more boxes.

## Agents with a more complex argument signature

In [ ]:
from langchain.tools import BaseTool

In [ ]:
import numpy as np
class NumGenerator():
    def __init__(self, added):
        self.added = added
    def make(self, seed):
        np.random.seed(seed)
        return np.random.randint(20) + self.added


class ComplexArgumentTool(BaseTool):
    name = "find_secret"
    description = "This is the tool to get the secret string corresponding to a given letter and a given integer seed."
    generator: NumGenerator = Field(exclude=True)
    
    def _run(self, letter: str, seed: int, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        shifted_letter = chr(ord('A')+((ord(letter[0].upper())-ord('A')+25) % 26))
        return shifted_letter * self.generator.make(seed)
    
    async def _arun(self, letter: str, seed: int, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("no async")


In [ ]:
generator = NumGenerator(4)
tool = ComplexArgumentTool(generator=generator)

In [ ]:
agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
agent_executor("""
    You are an assistant tasked with helping users to find their secret,
    and you can use the provided tools to achieve that goal.
    
    The secret is a deterministic function of a seed and a starting letter.
    Do not expect any clear relation between the inputs and the secret.
    
    The secret will be a certain letter repeated a certain number of times.

    My seed is eleven and my letter is a Q. Tell me my secret.
""")